### Importing The Libraries


In [1]:
import numpy as np
import pickle
import random
import yaml

import nltk 
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')


### Preprocessing Dataset

In [2]:
words=[]
classes = []
documents = []

ignore_words = ['?','!']

data_file = open('./data/intents.yaml').read()
intents = yaml.safe_load(data_file)

for intent in intents['intent']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intentsr
print (len(documents), "documents")
print("_" * 80)

# classes = intents
print ("Num of Intents",len(classes),'\n',classes)
print("_" * 80)

# words =  vocabulary
print (len(words), "unique words")

548 documents
________________________________________________________________________________
Num of Intents 13 
 ['Cooking', 'Datetime', 'Goodbye', 'Greeting', 'Info_Request', 'Iot', 'Joke', 'News', 'PlayMusic', 'Reminder', 'Thanks', 'Transport', 'Weather']
________________________________________________________________________________
517 unique words


### Create Training Data

In [3]:
# create our training data
train = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    train.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(train)
train = np.array(train,dtype=object)

# create train and test lists. X - patterns, Y - intents
train_x = list(train[:,0])
train_y = list(train[:,1])
print("Training data created")

Training data created


### Model Preparation & Training

In [4]:
# Create model - 3 layers. 
# 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent 
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [5]:
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
110/110 [==============================] - 2s 1ms/step - loss: 2.0780 - accuracy: 0.3193
Epoch 2/200
110/110 [==============================] - 0s 2ms/step - loss: 1.3821 - accuracy: 0.5730
Epoch 3/200
110/110 [==============================] - 0s 1ms/step - loss: 0.9806 - accuracy: 0.7208
Epoch 4/200
110/110 [==============================] - 0s 1ms/step - loss: 0.7915 - accuracy: 0.7518
Epoch 5/200
110/110 [==============================] - 0s 1ms/step - loss: 0.6211 - accuracy: 0.8212
Epoch 6/200
110/110 [==============================] - 0s 1ms/step - loss: 0.4800 - accuracy: 0.8613
Epoch 7/200
110/110 [==============================] - 0s 1ms/step - loss: 0.3627 - accuracy: 0.8942
Epoch 8/200
110/110 [==============================] - 0s 1ms/step - loss: 0.3360 - accuracy: 0.8978
Epoch 9/200
110/110 [==============================] - 0s 1ms/step - loss: 0.2860 - accuracy: 0.9106
Epoch 10/200
110/110 [==============================] - 0s 1ms/step - loss: 0.2692 - accura

110/110 [==============================] - 0s 1ms/step - loss: 0.0382 - accuracy: 0.9891
Epoch 82/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0205 - accuracy: 0.9945
Epoch 83/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 0.9964
Epoch 84/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0289 - accuracy: 0.9945
Epoch 85/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 0.9964
Epoch 86/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0286 - accuracy: 0.9909
Epoch 87/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0270 - accuracy: 0.9909
Epoch 88/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0226 - accuracy: 0.9945
Epoch 89/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 0.9982
Epoch 90/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0109 - accuracy: 

110/110 [==============================] - 0s 1ms/step - loss: 0.0122 - accuracy: 0.9964
Epoch 161/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 0.9945
Epoch 162/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0086 - accuracy: 0.9982
Epoch 163/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0129 - accuracy: 0.9945
Epoch 164/200
110/110 [==============================] - 0s 2ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 165/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 166/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0121 - accuracy: 0.9964
Epoch 167/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 168/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0085 - accuracy: 0.9982
Epoch 169/200
110/110 [==============================] - 0s 1ms/step - loss: 0.0035 - a

### Saving Model, Classes and Words

In [6]:
model.save('Model.h5',history)

with open('classes.pkl','wb') as file:  
    pickle.dump(classes,file)

with open('words.pkl','wb') as file:  
    pickle.dump(words,file)